In [29]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from torchvision.transforms import ToTensor, Resize
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, Dataset, DataLoader
from torch import optim
from PIL import Image
%matplotlib inline
import os

In [30]:
# for dirname, _, filenames in os.walk('input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [31]:
# train_data_dir = "input/train/train2500/"
# test_data_dir = "input/test/test1250/"
train_data_dir = "input_large/train/"
test_data_dir = "input_large/test1/"

In [32]:
class dataset_cat_dog(Dataset):
    def __init__(self, train_dir, transform=None):
        self.train_dir = train_dir
        self.transform = transform
        self.images = os.listdir(train_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image_path = os.path.join(self.train_dir, self.images[index])
        label = self.images[index].split(".")[0]
        label = 0 if label == 'cat' else 1

        image = Image.open(image_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        return image, label


In [33]:
transform = transforms.Compose([
    ToTensor(),
    Resize((256,256), antialias=None)
])
val_size = 0.2 
train_data = dataset_cat_dog(train_data_dir, transform)

val_size = int(val_size * len(train_data))
train_size = len(train_data) - val_size
train_data, val_data = random_split(train_data, [train_size, val_size])

train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
val_dl = DataLoader(val_data, batch_size=32, shuffle=False)

# train_data = dataset_cat_dog(train_data_dir, transform)
# val_data = dataset_cat_dog(test_data_dir, transform)

# train_dl = DataLoader(train_data, batch_size=32, shuffle=True)
# val_dl = DataLoader(val_data, batch_size=32)

In [34]:
# Size of the datasets
print("Train Data Size:", len(train_data))
print("Validation Data Size:", len(val_data))


Train Data Size: 20000
Validation Data Size: 5000


In [35]:
# img, label = train_data[0]
# print(img.shape,label)

# def PrintImages():
#     train_data = CatDogDataset(train_data_dir, transform)
#     c = 0
#     d = 0

#     # Iterate over the dataset to see the images and labels
#     for i in range(len(train_data)):
#         image, label = train_data[i]

#         # Print the label
#         if label == 0 and c <= 10:
#             c += 1
#             print("Label: Cat - ", c)
#         elif label == 1 and d <= 10:
#             d += 1
#             print("Label: Dog - ", d)
#         else:
#             continue

#         # Transpose the image dimensions from (3, 150, 150) to (150, 150, 3)
#         image = np.transpose(image, (1, 2, 0))

#         # Display the image
#         plt.imshow(image)
#         plt.axis('off')
#         plt.show()

#         print("-----------------------")

# PrintImages()

In [36]:
# print(f"Images in tarining data : {len(train_dataset)}")
# print(f"Images in test data : {len(test_dataset)}")

In [37]:
# print("Follwing classes are there : \n",train_dataset.classes)

In [38]:
# from torchvision.utils import make_grid

# def show_batch(dl):
#     for images, labels in dl:
#         fig,ax = plt.subplots(figsize = (16,12))
#         ax.set_xticks([])
#         ax.set_yticks([])
#         ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
#         break

# show_batch(train_dl)

In [39]:
# # Define the CNN architecture
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.relu1 = nn.ReLU()
#         self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
#         self.bn2 = nn.BatchNorm2d(32)
#         self.relu2 = nn.ReLU()
#         self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(32 * 64 * 64, 128)
#         self.relu3 = nn.ReLU()
#         # self.dropout = nn.Dropout(0.5)  # Dropout with 50% probability
#         self.fc2 = nn.Linear(128, 2)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu1(x)
#         x = self.pool1(x)
#         x = self.conv2(x)
#         x = self.bn2(x)
#         x = self.relu2(x)
#         x = self.pool2(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc1(x)
#         x = self.relu3(x)
#         # x = self.dropout(x)
#         x = self.fc2(x)
#         return x

In [40]:
# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Convolutional layers for feature extraction
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        # self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        # self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        
        # Fully connected layers for classification
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        # self.dropout = nn.Dropout(0.5)  # Dropout with 50% probability
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        # x = x.view(-1, 64 * 64 * 64)  # Flatten the output from convolutional layers
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [41]:
# Initialize the CNN model
model = CNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=65536, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [42]:
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    val_loss = 0.0
    val_correct = 0

    # Training
    model.train()
    for images, labels in train_dl:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        train_loss += loss.item() * images.size(0)
        train_correct += torch.sum(preds == labels.data)

    train_loss = train_loss / len(train_data)
    train_acc = train_correct.double() / len(train_data)

    # Validation
    model.eval()
    with torch.no_grad():
        for images, labels in val_dl:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * images.size(0)
            val_correct += torch.sum(preds == labels.data)

    val_loss = val_loss / len(val_data)
    val_acc = val_correct.double() / len(val_data)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val Acc: {val_acc:.4f}")
    print("--------------------------------------")


Epoch 1/15
Train Loss: 0.6445
Train Acc: 0.6227
Val Loss: 0.5871
Val Acc: 0.6998
--------------------------------------
Epoch 2/15
Train Loss: 0.5653
Train Acc: 0.7081
Val Loss: 0.5357
Val Acc: 0.7320
--------------------------------------
Epoch 3/15
Train Loss: 0.5283
Train Acc: 0.7313
Val Loss: 0.5091
Val Acc: 0.7530
--------------------------------------
Epoch 4/15
Train Loss: 0.5023
Train Acc: 0.7530
Val Loss: 0.5422
Val Acc: 0.7124
--------------------------------------
Epoch 5/15
Train Loss: 0.4769
Train Acc: 0.7730
Val Loss: 0.4982
Val Acc: 0.7544
--------------------------------------
Epoch 6/15
Train Loss: 0.4543
Train Acc: 0.7858
Val Loss: 0.4905
Val Acc: 0.7598
--------------------------------------
Epoch 7/15
Train Loss: 0.4334
Train Acc: 0.8000
Val Loss: 0.4585
Val Acc: 0.7830
--------------------------------------
Epoch 8/15
Train Loss: 0.4132
Train Acc: 0.8101
Val Loss: 0.4570
Val Acc: 0.7802
--------------------------------------
Epoch 9/15
Train Loss: 0.3950
Train Acc:

In [43]:
# Save the trained model
torch.save(model.state_dict(), "cat_dog_model.pth")

In [44]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from PIL import Image

# Load the saved model state dictionary
model = CNN()
model.load_state_dict(torch.load("cat_dog_model.pth"))
model.eval()

# Define the transformation for the new images
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Load and preprocess a new image
image_path = "input_large/test1/200.jpg"
image = Image.open(image_path).convert("RGB")
image = transform(image)
image = image.unsqueeze(0)

# Make a prediction on the new image
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output.data, 1)

# Map the predicted label to the corresponding class name
class_names = ["cat", "dog"]
predicted_class = class_names[predicted.item()]

print(f"Predicted class: {predicted_class}")


Predicted class: cat
